# ValidationPipeline Tutorial

The **ValidationPipeline** demonstrates multi-step workflows using LangGraph - the 15% use case for more complex validation scenarios.

## Overview

- **Type**: LangGraph Workflow (multi-step)
- **Use Case**: Complex validation requiring multiple passes
- **Features**: State management, step chaining, customizable prompts per step

## Workflow Steps

```
+---------------+     +------------------+     +--------+
| 1. Quick Scan | --> | 2. Detailed      | --> |  END   |
|   (fast)      |     |    Verify (deep) |     |        |
+---------------+     +------------------+     +--------+
```

## Setup

In [ ]:
from agent_workshop.agents.pipelines import ValidationPipeline
from agent_workshop import Config

config = Config()
pipeline = ValidationPipeline(config)

print(f"Provider: {pipeline.provider_name}")
print(f"Model: {pipeline.model_name}")

## Input/Output Format

**Input**: Dictionary with content field
```python
{
    "content": str  # Document content to validate
}
```

**Output**: Dictionary with results from each step
```python
{
    "content": str,           # Original content
    "scan_result": str,       # Quick scan findings
    "verify_result": str,     # Detailed verification
    "final_result": {
        "quick_scan": str,
        "detailed_verification": str,
        "workflow_complete": bool
    }
}
```

## Understanding the Steps

### Step 1: Quick Scan
A fast preliminary check that looks for:
- Format/structure issues
- Missing critical sections
- Obvious errors or red flags

### Step 2: Detailed Verify
An in-depth validation that:
- Uses the quick scan results as context
- Provides specific recommendations
- Generates structured JSON output with status, strengths, recommendations

In [ ]:
# Inspect the default prompts
print("Quick Scan Prompt (first 200 chars):")
print(pipeline.quick_scan_prompt[:200] + "...")

print("\nDetailed Verify Prompt (first 200 chars):")
print(pipeline.detailed_verify_prompt[:200] + "...")

## Example: Validate a Document

In [ ]:
# Sample document with some issues
sample_doc = '''
Project Status Report - Sprint 15

## Completed
- User authentication module
- Database migration scripts

## In Progress
- API documentation
- Performance testing

## Blockers
- Waiting on security team review

Team velocity: 42 points
'''

print("Document to validate:")
print(sample_doc)

In [ ]:
# Run the pipeline (uncomment to execute)
# result = await pipeline.run({"content": sample_doc})
#
# print("Quick Scan Result:")
# print("=" * 50)
# print(result.get('scan_result'))
#
# print("\nDetailed Verification:")
# print("=" * 50)
# print(result.get('verify_result'))

## State Management

LangGraph workflows use a TypedDict to manage state between steps:

```python
class ValidationState(TypedDict):
    content: str           # Input document
    scan_result: str | None    # After step 1
    verify_result: str | None  # After step 2
    final_result: dict | None  # Combined results
```

Each step receives the current state and returns an updated version.

In [ ]:
# View the workflow graph structure
graph = pipeline.build_graph()

print("Pipeline Nodes:")
for node in graph.nodes:
    if not node.startswith("__"):  # Skip internal nodes
        print(f"  - {node}")

## Custom Prompts

You can customize the prompts for each step:

In [ ]:
# Custom pipeline for technical specifications
tech_pipeline = ValidationPipeline(
    config=config,
    quick_scan_prompt="""Perform a QUICK SCAN of this technical specification.

Check for:
1. Missing architecture diagrams
2. Undefined technical terms
3. Incomplete API definitions

Content:
{content}

Provide a brief assessment.""",
    detailed_verify_prompt="""Based on the quick scan, perform DETAILED VERIFICATION.

Quick Scan Results:
{scan_result}

Original Spec:
{content}

Provide:
1. Completeness score (1-10)
2. Technical accuracy assessment
3. Specific gaps to address
4. Priority recommendations"""
)

print("Custom Technical Spec Pipeline Created")

## YAML Configuration

Configure the pipeline using a YAML file:

```yaml
# prompts.yaml
validation_pipeline:
  quick_scan_prompt: |
    Perform a quick scan of this content:
    {content}
    
    Look for obvious issues.

  detailed_verify_prompt: |
    Based on scan results: {scan_result}
    
    Verify: {content}
    
    Provide detailed recommendations.
```

```python
pipeline = ValidationPipeline(config, config_file="prompts.yaml")
```

## When to Use Multi-Step Pipelines

Choose ValidationPipeline over DeliverableValidator when you need:

| Requirement | Simple Agent | Pipeline |
|-------------|--------------|----------|
| Single-pass validation | Best | Overkill |
| Multi-stage analysis | Limited | Best |
| Results from step inform next | No | Yes |
| Conditional routing | No | Possible |
| Complex state management | No | Yes |

## Adding Custom Steps

To extend the pipeline with additional steps, subclass `LangGraphAgent`:

```python
from agent_workshop.workflows import LangGraphAgent
from langgraph.graph import StateGraph, END

class ExtendedPipeline(LangGraphAgent):
    def build_graph(self):
        workflow = StateGraph(dict)
        
        # Add custom steps
        workflow.add_node("step1", self.step1)
        workflow.add_node("step2", self.step2)
        workflow.add_node("step3", self.step3)  # New step!
        
        # Define flow
        workflow.add_edge("step1", "step2")
        workflow.add_edge("step2", "step3")
        workflow.add_edge("step3", END)
        
        workflow.set_entry_point("step1")
        return workflow.compile()
```

## Next Steps

- **[04_pr_pipeline.ipynb](./04_pr_pipeline.ipynb)** - Three-step PR review workflow
- **[05_release_pipeline.ipynb](./05_release_pipeline.ipynb)** - Hybrid workflow with shell commands
- **[01_deliverable_validator.ipynb](./01_deliverable_validator.ipynb)** - Simple validation agent